# Sentiment causal analyses based on movie reviews (topic modeling)
## Based on 50,000 labeled IMDb movie reviews [dataset](http://ai.stanford.edu/~amaas/data/sentiment/)

# Import necessary dependencies

In [1]:
import pandas as pd
import numpy as np

# Python files with functions
import text_normalizer as tn
import model_evaluation_utils as meu
# ~Python files

np.set_printoptions(precision=2, linewidth=80)

# Load, split (train, test) and normalize the data

In [3]:
# Use subset of the data for better performance
train_size = 500
test_size = 500

In [4]:
dataset = pd.read_csv('movie_reviews.csv').sample(frac=1).reset_index(
    drop=True)

reviews = np.array(dataset['review'])
sentiments = np.array(dataset['sentiment'])

# train sets
train_reviews = reviews[:train_size]
train_sentiments = sentiments[:train_size]

# test sets
test_reviews = reviews[train_size:train_size + test_size]
test_sentiments = sentiments[train_size:train_size + test_size]

# normalized reviews
norm_train_reviews = tn.normalize_corpus(train_reviews)
norm_test_reviews = tn.normalize_corpus(test_reviews)

# peek at data
print(dataset.head(1))

                                              review sentiment
0  There is so much not to like about this show i...  negative


# Extract features from positive and negative reviews

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

# consolidate all normalized reviews
norm_reviews = norm_train_reviews + norm_test_reviews
# get tfidf features for positive reviews
positive_reviews = [
    review for review, sentiment in zip(norm_reviews, sentiments)
    if sentiment == 'positive'
]
ptvf = TfidfVectorizer(
    min_df=0.05,
    max_df=0.95,
    ngram_range=(1, 1),
    sublinear_tf=True,
    use_idf=True,
    binary=False)
ptvf_features = ptvf.fit_transform(positive_reviews)
# get tfidf features for negative reviews
negative_reviews = [
    review for review, sentiment in zip(norm_reviews, sentiments)
    if sentiment == 'negative'
]
ntvf = TfidfVectorizer(
    min_df=0.05,
    max_df=0.95,
    ngram_range=(1, 1),
    sublinear_tf=True,
    use_idf=True,
    binary=False)
ntvf_features = ntvf.fit_transform(negative_reviews)

print('Positive features shape: ', ptvf_features.shape,
      ' Negative features shape: ', ntvf_features.shape)

Positive features shape:  (498, 342)  Negative features shape:  (502, 337)


+ min_df and max_df parameters were set to get rid of features that occur too often or too rarely and to speed up the process

In [13]:
import pyLDAvis # for building interactive visualizations of topic models
import pyLDAvis.sklearn
from sklearn.decomposition import NMF
import topic_model_utils as tmu

pyLDAvis.enable_notebook()
total_topics = 10

## Display and visualize topics for positive reviews

In [66]:
def my_display_topics(weights_matrix, pos_feature_names, words=15):
    weights_to_names = [list(zip(weights,np.array(pos_feature_names).flatten())) for weights in weights_matrix]
    weights_to_names = [sorted(row,key= lambda x: x[0],reverse=True) for row in weights_to_names]
    for row in weights_to_names:
        stacked_weights_to_names_row = np.vstack(row)
        print(stacked_weights_to_names_row[:words,1])

In [58]:
# build topic model on positive sentiment review features
pos_nmf = NMF(n_components=total_topics, 
          random_state=42, alpha=0.1, l1_ratio=0.2)
pos_nmf.fit(ptvf_features)
# extract features and component weights
pos_feature_names = ptvf.get_feature_names()
pos_weights = pos_nmf.components_

# extract and display topics and their components
pos_topics = tmu.get_topics_terms_weights(pos_weights, pos_feature_names)
tmu.print_topics_udf(topics=pos_topics,
                 total_topics=total_topics,
                 num_terms=15,
                 display_weights=False)
# or

# my_display_topics(pos_weights,pos_feature_names)

Topic #1 without weights
['man', 'life', 'young', 'no', 'come', 'woman', 'know', 'take', 'become', 'one', 'family', 'get', 'kill', 'new', 'wife']

Topic #2 without weights
['movie', 'see', 'watch', 'good', 'not', 'go', 'like', 'time', 'want', 'think', 'get', 'never', 'one', 'year', 'first']

Topic #3 without weights
['role', 'play', 'performance', 'well', 'film', 'cast', 'good', 'excellent', 'give', 'actor', 'year', 'also', 'great', 'superb', 'work']

Topic #4 without weights
['series', 'episode', 'show', 'tv', 'watch', 'not', 'dvd', 'first', 'something', 'air', 'great', 'get', 'boy', 'everyone', 'could']

Topic #5 without weights
['funny', 'comedy', 'laugh', 'character', 'make', 'show', 'not', 'movie', 'great', 'good', 'support', 'enjoy', 'girl', 'script', 'lot']

Topic #6 without weights
['character', 'story', 'interesting', 'not', 'beautiful', 'wonderful', 'give', 'rather', 'something', 'relationship', 'viewer', 'lead', 'also', 'life', 'actor']

Topic #7 without weights
['love', 'fi

In [55]:
pyLDAvis.sklearn.prepare(pos_nmf, ptvf_features, ptvf, R=15)

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation a

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.007372  0.048745       1        1  15.079970
8      0.040308 -0.046773       2        1  12.213181
1      0.146869 -0.026078       3        1  11.684346
2     -0.096959  0.091947       4        1  11.334617
7     -0.064133  0.074461       5        1  11.027544
5     -0.190151  0.071991       6        1  10.365517
6      0.120257 -0.037408       7        1   8.170556
9     -0.019530  0.043120       8        1   7.036698
4     -0.104496 -0.276945       9        1   6.601517
3      0.160463  0.056940      10        1   6.486053, topic_info=     Category       Freq       Term      Total  loglift  logprob
term                                                            
184   Default  24.000000       love  24.000000  15.0000  15.0000
200   Default  44.000000      movie  44.000000  14.0000  14.0000
122   Default  16.000000      funny  16.000000  13.0000  13.0000
267   Default  15.000000     series  15.000000  12.0000  12.0000
57    Default  14.000000     comedy  14.000000  11.0000  11.0000
148   Default  13.000000     horror  13.000000  10.0000  10.0000
51    Default  23.000000  character  23.000000   9.0000   9.0000
111   Default  39.000000       film  39.000000   8.0000   8.0000
288   Default  20.000000      story  20.000000   7.0000   7.0000
320   Default  12.000000        war  12.000000   6.0000   6.0000
86    Default   9.000000    episode   9.000000   5.0000   5.0000
168   Default  11.000000      laugh  11.000000   4.0000   4.0000
256   Default  12.000000       role  12.000000   3.0000   3.0000
263   Default  31.000000        see  31.000000   2.0000   2.0000
271   Default  19.000000       show  19.000000   1.0000   1.0000
70     Topic1   3.823755        die   4.056724   1.8327  -4.7460
198    Topic1   3.318837     mother   4.089926   1.6829  -4.8876
105    Topic1   3.244494     father   4.027046   1.6757  -4.9103
325    Topic1   4.403869       wife   5.508043   1.6681  -4.6047
32     Topic1   5.592407     become   7.106190   1.6522  -4.3658
329    Topic1   6.171702      woman   7.849462   1.6513  -4.2673
341    Topic1   7.328560      young   9.390388   1.6439  -4.0954
162    Topic1   4.469555       kill   6.160351   1.5710  -4.5899
167    Topic1   3.931208      later   5.753092   1.5110  -4.7183
146    Topic1   3.320993       home   4.876245   1.5077  -4.8870
43     Topic1   2.489658    brother   3.772773   1.4761  -5.1751
44     Topic1   3.134890       call   5.019687   1.4210  -4.9446
187    Topic1   7.946013        man  12.805553   1.4146  -4.0146
193    Topic1   2.444417       meet   3.989351   1.4020  -5.1934
136    Topic1   2.496780       head   4.127114   1.3892  -5.1722
...       ...        ...        ...        ...      ...      ...
271    Topic9   3.693345       show  19.062330   1.0767  -3.9546
186    Topic9   3.852246       make  23.714644   0.9004  -3.9125
183    Topic9   2.537323        lot  12.189086   1.1484  -4.3300
129    Topic9   3.042338      great  23.775184   0.6619  -4.1485
213    Topic9   3.556976        not  41.164410   0.2692  -3.9922
200    Topic9   3.509608      movie  44.185560   0.1850  -4.0057
128    Topic9   2.998744       good  31.635547   0.3618  -4.1630
267   Topic10  15.844633     series  15.844633   2.7355  -2.4807
86    Topic10   9.374152    episode   9.583488   2.7134  -3.0056
311   Topic10   5.757231         tv   8.179917   2.3843  -3.4931
10    Topic10   2.513815        air   4.708461   2.1080  -4.3217
75    Topic10   3.057853        dvd   6.847012   1.9294  -4.1258
271   Topic10   8.141430       show  19.062330   1.8848  -3.1465
47    Topic10   1.733166      catch   4.715123   1.7347  -4.6936
39    Topic10   2.294782        boy   6.747660   1.6570  -4.4129
257   Topic10   1.918103        run   5.989343   1.5969  -4.5922
251   Topic10   1.473975       rest   4.739493   1.5675  -4.8556
317   Topic10   1.754535       view   6.548714   1.4185  -4

## Display and visualize topics for negative reviews

In [64]:
# build topic model on positive sentiment review features
neg_nmf = NMF(n_components=total_topics, 
          random_state=42, alpha=0.1, l1_ratio=0.2)
neg_nmf.fit(ntvf_features)
# extract features and component weights
neg_feature_names = ntvf.get_feature_names()
neg_weights = neg_nmf.components_

# extract and display topics and their components
neg_topics = tmu.get_topics_terms_weights(neg_weights, neg_feature_names)
tmu.print_topics_udf(topics=neg_topics,
                 total_topics=total_topics,
                 num_terms=15,
                 display_weights=False)
# or

# my_display_topics(neg_weights,neg_feature_names)

Topic #1 without weights
['film', 'may', 'minute', 'though', 'two', 'come', 'lead', 'well', 'one', 'time', 'start', 'around', 'man', 'director', 'take']

Topic #2 without weights
['movie', 'bad', 'could', 'watch', 'good', 'not', 'see', 'say', 'ever', 'first', 'make', 'thing', 'even', 'think', 'get']

Topic #3 without weights
['show', 'kid', 'back', 'guy', 'get', 'go', 'watch', 'old', 'think', 'day', 'want', 'look', 'use', 'really', 'like']

Topic #4 without weights
['life', 'story', 'make', 'man', 'movie', 'not', 'love', 'many', 'hour', 'time', 'get', 'family', 'know', 'no', 'like']

Topic #5 without weights
['character', 'really', 'seem', 'like', 'lot', 'not', 'care', 'version', 'scene', 'people', 'main', 'much', 'find', 'also', 'lack']

Topic #6 without weights
['original', 'series', 'sequel', 'story', 'nothing', 'fan', 'part', 'credit', 'never', 'already', 'good', 'still', 'even', 'line', 'least']

Topic #7 without weights
['comedy', 'joke', 'funny', 'laugh', 'bad', 'not', 'woman', 

In [63]:
pyLDAvis.sklearn.prepare(neg_nmf, ntvf_features, ntvf, R=15)

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation a

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.005547 -0.004706       1        1  16.675120
1     -0.018166  0.070391       2        1  14.516675
4      0.072568 -0.004747       3        1  11.331317
2      0.059689 -0.013900       4        1  11.143641
8     -0.088007  0.181236       5        1   9.893735
9     -0.123977 -0.169321       6        1   8.832159
3     -0.028322 -0.142890       7        1   8.474521
7      0.167022  0.145986       8        1   7.097807
6      0.170325 -0.116520       9        1   6.017929
5     -0.205584  0.054471      10        1   6.017096, topic_info=     Category       Freq       Term      Total  loglift  logprob
term                                                            
107   Default  43.000000       film  43.000000  15.0000  15.0000
210   Default  15.000000   original  15.000000  14.0000  14.0000
167   Default  17.000000       life  17.000000  13.0000  13.0000
58    Default  14.000000     comedy  14.000000  12.0000  12.0000
81    Default  15.000000     effect  15.000000  11.0000  11.0000
276   Default  12.000000    special  12.000000  10.0000  10.0000
284   Default  24.000000      story  24.000000   9.0000   9.0000
5     Default  17.000000      actor  17.000000   8.0000   8.0000
150   Default  12.000000       joke  12.000000   7.0000   7.0000
192   Default  42.000000      movie  42.000000   6.0000   6.0000
27    Default  29.000000        bad  29.000000   5.0000   5.0000
117   Default  14.000000      funny  14.000000   4.0000   4.0000
161   Default  13.000000      laugh  13.000000   3.0000   3.0000
255   Default  30.000000        see  30.000000   2.0000   2.0000
53    Default  21.000000  character  21.000000   1.0000   1.0000
129    Topic1   3.447100       hand   3.554071   1.7607  -4.9881
32     Topic1   2.997893     behind   3.630736   1.5997  -5.1277
162    Topic1   4.440818       lead   5.667315   1.5474  -4.7348
132    Topic1   3.473614       head   4.445659   1.5445  -4.9804
261    Topic1   3.641607        set   5.014115   1.4714  -4.9332
97     Topic1   3.165810       face   4.446683   1.4515  -5.0732
160    Topic1   2.833852      later   4.240611   1.3882  -5.1840
9      Topic1   2.580683      along   4.118896   1.3237  -5.2776
189    Topic1   2.733800       miss   4.488051   1.2955  -5.2199
182    Topic1   6.186940        may  10.395144   1.2724  -4.4032
69     Topic1   2.367905      death   4.017772   1.2625  -5.3636
232    Topic1   3.366153     rather   5.731833   1.2590  -5.0119
104    Topic1   2.530598    feature   4.309377   1.2589  -5.2972
46     Topic1   3.615771     camera   6.191359   1.2534  -4.9403
297    Topic1   5.188817     though   8.922472   1.2492  -4.5791
...       ...        ...        ...        ...      ...      ...
2      Topic9   2.484042        act  11.951791   1.2394  -4.2966
27     Topic9   4.651212        bad  29.172344   0.9743  -3.6693
204    Topic9   4.440542        not  45.705807   0.4790  -3.7157
180    Topic9   2.657777        man  14.006505   1.1484  -4.2290
53     Topic9   2.697731  character  21.325292   0.7429  -4.2141
91     Topic9   2.463101       even  23.502477   0.5547  -4.3050
92     Topic9   2.332507       ever  16.663130   0.8442  -4.3595
210   Topic10  15.404655   original  15.404655   2.8106  -2.4717
258   Topic10   6.204352     sequel   7.500678   2.6208  -3.3811
260   Topic10   8.173445     series   9.892354   2.6197  -3.1054
10    Topic10   2.805834    already   4.860971   2.2610  -4.1746
65    Topic10   3.095285     credit   5.735421   2.1938  -4.0764
102   Topic10   3.591751        fan   7.876467   2.0253  -3.9277
252   Topic10   2.316944     screen   6.718702   1.7459  -4.3661
298   Topic10   2.304558      three   6.953676   1.7062  -4.3714
325   Topic10   1.756160     wonder   5.426418   1.6824  -4.6432
230   Topic10   2.040000    quality   6.306756   1.6819  -4.4934
187   Topic10   1.457625       mind   4.656456   1.6491  -4